In [58]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime, timedelta
import time

In [59]:
# create a function to categorize the time into periods
def categorize_time(time):
    if time.hour < 9 or (time.hour == 9 and time.minute < 30):
        return 'pre-market'
    elif 9 <= time.hour < 16 or (time.hour == 9 and time.minute >= 30):
        return 'intraday'
    else:
        return 'aftermarket'

In [60]:
# Pull in the list of companies

df_SP_500 = pd.read_csv("constituents.csv")
ticker_list = df_SP_500['Symbol'].astype(str).tolist()
ticker_list = [t.replace('.', '-') for t in ticker_list]

for ticker in ticker_list:
    print(ticker)

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BRK-B
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BSX
BMY
AVGO
BR
BRO
BF-B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ

In [61]:
# create the placeholder data frame
holding_df = pd.DataFrame()

for ticker in ticker_list:
    # Read the CSV file into a DataFrame
    df_test = pd.read_csv(f'Media_Sentiment_Scores\{ticker}_Sentiments.csv')

    # Convert 'time_published' to datetime
    df_test['time_published'] = pd.to_datetime(df_test['time_published'])

    # Add a new column that creates a relevance weighted sentiment score
    df_test['relevance_weighted_sentiment_score'] = df_test['relevance_score'] * df_test['ticker_sentiment_score']

    # Apply the time periods function
    df_test['time_period'] = df_test['time_published'].apply(categorize_time)

    # Define the impacted trading day
    df_test['date'] = df_test['time_published'] + timedelta(hours=8)
    df_test['date'] = df_test['date'].dt.date

    df_test = df_test.drop(['title','url','time_published','summary','ticker','relevance_score','ticker_sentiment_label'], axis=1)

    # # Display the DataFrame
    # display(df_test)

    # Group by 'trading_date' and 'time_period' and calculate mean sentiment scores
    grouped = df_test.groupby(['date', 'time_period']).agg({
                'ticker_sentiment_score': 'mean',
                'relevance_weighted_sentiment_score': 'mean'
                }).reset_index()

    # Pivot the DataFrame
    pivoted = grouped.pivot(index='date', columns='time_period', values=['ticker_sentiment_score', 'relevance_weighted_sentiment_score'])
    pivoted.columns = [f'{col[0]}_{col[1]}' for col in pivoted.columns]
    pivoted.reset_index(inplace=True)
    pivoted.fillna(0, inplace=True)

    # display(pivoted)

    # Read the CSV file into a DataFrame
    df_test_too = pd.read_csv(f'2023_Closing_Stock_Prices\{ticker}_stock_data.csv')

    # Convert 'time_published' to datetime
    df_test_too['date'] = pd.to_datetime(df_test_too['date'])
    pivoted['date'] = pd.to_datetime(pivoted['date'])

    # Perform a left merge
    merged_df = df_test_too.merge(pivoted, on='date', how='left')
    merged_df.reset_index(drop=True, inplace=True)
    merged_df.fillna(0, inplace=True)

    # display(merged_df)

    sector = df_SP_500.loc[df_SP_500["Symbol"] == ticker, "GICS Sector"].values[0]
    merged_df['GICS Sector'] = sector

    industry = df_SP_500.loc[df_SP_500["Symbol"] == ticker, "GICS Sub-Industry"].values[0]
    merged_df['GICS Sub-Industry'] = industry

    # display(merged_df)
    holding_df = pd.concat([holding_df, merged_df], ignore_index=True)

file_path = 'Final_File/combined.csv'
os.makedirs(os.path.dirname(file_path), exist_ok=True)
holding_df.to_csv(file_path, index=False)

print(f"DataFrame exported to {file_path}")
    

EmptyDataError: No columns to parse from file

In [44]:
# Group by 'trading_date' and 'time_period' and calculate mean sentiment scores
grouped = df_test.groupby(['date', 'time_period']).agg({
    'ticker_sentiment_score': 'mean',
    'relevance_weighted_sentiment_score': 'mean'
}).reset_index()

# Pivot the DataFrame
pivoted = grouped.pivot(index='date', columns='time_period', values=['ticker_sentiment_score', 'relevance_weighted_sentiment_score'])
pivoted.columns = [f'{col[0]}_{col[1]}' for col in pivoted.columns]
pivoted.reset_index(inplace=True)
pivoted.fillna(0, inplace=True)


display(pivoted)

,date,ticker_sentiment_score_aftermarket,ticker_sentiment_score_intraday,ticker_sentiment_score_pre-market,relevance_weighted_sentiment_score_aftermarket,relevance_weighted_sentiment_score_intraday,relevance_weighted_sentiment_score_pre-market
0,2023-01-02,0.000000,-0.188237,0.000000,0.000000,-0.056373,0.000000
1,2023-01-03,0.000000,0.027710,0.000000,0.000000,0.007648,0.000000
2,2023-01-12,0.245761,0.000000,0.000000,0.036217,0.000000,0.000000
3,2023-01-13,0.302525,0.000000,0.000000,0.062262,0.000000,0.000000
4,2023-01-14,0.037005,0.000000,0.000000,0.002005,0.000000,0.000000
...,...,...,...,...,...,...,...
216,2024-01-24,-0.059805,-0.100818,0.525561,-0.010489,-0.011673,0.121579
217,2024-01-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
218,2024-01-26,0.352221,0.000000,0.000000,0.020545,0.000000,0.000000
219,2024-01-28,0.000000,0.091281,0.000000,0.000000,0.006253,0.000000


In [47]:
# Read the CSV file into a DataFrame
df_test_too = pd.read_csv('2023_Closing_Stock_Prices\MMM_stock_data.csv')

# Convert 'time_published' to datetime
df_test_too['date'] = pd.to_datetime(df_test_too['date'])
pivoted['date'] = pd.to_datetime(pivoted['date'])

# Perform a left merge
merged_df = df_test_too.merge(pivoted, on='date', how='left')
merged_df.reset_index(drop=True, inplace=True)
merged_df.fillna(0, inplace=True)

display(merged_df)

,date,close,volume,close_1_day_prior,close_1_week_prior,pct_change_1_day,pct_change_1_week,ticker_sentiment_score_aftermarket,ticker_sentiment_score_intraday,ticker_sentiment_score_pre-market,relevance_weighted_sentiment_score_aftermarket,relevance_weighted_sentiment_score_intraday,relevance_weighted_sentiment_score_pre-market
0,2023-12-29,109.32,2414408,109.69,105.56,-0.003373,0.035620,0.0,-0.133600,0.0,0.0,-0.033975,0.0
1,2023-12-28,109.69,2809576,108.74,103.67,0.008736,0.058069,0.0,0.000000,0.0,0.0,0.000000,0.0
2,2023-12-27,108.74,2428836,108.11,106.25,0.005827,0.023435,0.0,0.000000,0.0,0.0,0.000000,0.0
3,2023-12-26,108.11,2786348,106.33,105.87,0.016740,0.021158,0.0,0.000000,0.0,0.0,0.000000,0.0
4,2023-12-22,106.33,2281685,105.56,106.98,0.007294,-0.006076,0.0,0.014477,0.0,0.0,0.001547,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2023-01-09,126.79,2871264,126.72,119.92,0.000552,0.057288,0.0,0.000000,0.0,0.0,0.000000,0.0
246,2023-01-06,126.72,2417142,122.96,120.57,0.030579,0.051008,0.0,0.000000,0.0,0.0,0.000000,0.0
247,2023-01-05,122.96,2606564,125.15,118.29,-0.017499,0.039479,0.0,0.000000,0.0,0.0,0.000000,0.0
248,2023-01-04,125.15,2769831,122.47,120.22,0.021883,0.041008,0.0,0.000000,0.0,0.0,0.000000,0.0


In [56]:
# Read the CSV file into a DataFrame
industry_label = pd.read_csv('constituents.csv')

sector = industry_label.loc[industry_label["Symbol"] == "MMM", "GICS Sector"].values[0]
merged_df['GICS Sector'] = sector

industry = industry_label.loc[industry_label["Symbol"] == "MMM", "GICS Sub-Industry"].values[0]
merged_df['GICS Sub-Industry'] = industry

display(merged_df)

,date,close,volume,close_1_day_prior,close_1_week_prior,pct_change_1_day,pct_change_1_week,ticker_sentiment_score_aftermarket,ticker_sentiment_score_intraday,ticker_sentiment_score_pre-market,relevance_weighted_sentiment_score_aftermarket,relevance_weighted_sentiment_score_intraday,relevance_weighted_sentiment_score_pre-market,GICS Sector,GICS Sub-Industry
0,2023-12-29,109.32,2414408,109.69,105.56,-0.003373,0.035620,0.0,-0.133600,0.0,0.0,-0.033975,0.0,Industrials,Industrial Conglomerates
1,2023-12-28,109.69,2809576,108.74,103.67,0.008736,0.058069,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
2,2023-12-27,108.74,2428836,108.11,106.25,0.005827,0.023435,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
3,2023-12-26,108.11,2786348,106.33,105.87,0.016740,0.021158,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
4,2023-12-22,106.33,2281685,105.56,106.98,0.007294,-0.006076,0.0,0.014477,0.0,0.0,0.001547,0.0,Industrials,Industrial Conglomerates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2023-01-09,126.79,2871264,126.72,119.92,0.000552,0.057288,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
246,2023-01-06,126.72,2417142,122.96,120.57,0.030579,0.051008,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
247,2023-01-05,122.96,2606564,125.15,118.29,-0.017499,0.039479,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
248,2023-01-04,125.15,2769831,122.47,120.22,0.021883,0.041008,0.0,0.000000,0.0,0.0,0.000000,0.0,Industrials,Industrial Conglomerates
